In [ ]:
### Check the correct virtual enviornment

import sys
sys.prefix

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv('GROQ_API_KEY')

In [ ]:
from langchain_groq import ChatGroq
model = ChatGroq(model='gemma2-9b-it', groq_api_key=groq_api_key)
model

### Documentation on Human , System Message

https://python.langchain.com/api_reference/core/messages.html

https://python.langchain.com/api_reference/core/messages/langchain_core.messages.system.SystemMessage.html

https://python.langchain.com/api_reference/core/messages/langchain_core.messages.human.HumanMessage.html

https://python.langchain.com/api_reference/core/messages/langchain_core.messages.ai.AIMessage.html

In [ ]:

from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content='Hi my name is Ankur and I am the chief AI engineer.')])

### Scenario 1 

Creating a simple chatbot where messages are passed as sequence and the chat model is able to remember the information provided

In [ ]:
from langchain_core.messages import AIMessage

In [ ]:
messages = [HumanMessage(content='Hi my name is Ankur and I am the chief AI engineer.'),
            AIMessage(content="Hello Ankur, it's nice to meet you! As a large language model, I'm always eager to learn from experienced AI engineers like yourself. \n\nWhat are you working on these days?  Is there anything I can help you with?\n"),
            HumanMessage("What is my name and what I do?")]

In [ ]:
model.invoke(messages)

### Scenario 2

- Message History

We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

Documentation for 


- ChatMessageHistory https://python.langchain.com/api_reference/community/chat_message_histories.html


- BaseChatMessageHistory   https://python.langchain.com/api_reference/core/chat_history/langchain_core.chat_history.BaseChatMessageHistory.html


- RunnableWithMessageHistory   https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory ### Chat message history stores a history of the message interactions in a chat.


from langchain_core.chat_history import BaseChatMessageHistory   ###  Abstract base class for storing chat message history.


from langchain_core.runnables.history import RunnableWithMessageHistory  ### RunnableWithMessageHistory wraps another Runnable and manages the chat message history for it; it is responsible for reading and updating the chat message history

When multiple users interact with the chat model how we will make sure that one session is completly different from another sesssion ?

We will create a function as below -

1) It accepts session ID as input. It will be used to distinguish one session from another
2) It returns the object of type BaseChatMessageHistory

In [ ]:
store = {}

### function that retrives chat history for a specific session id

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [ ]:
### Interact with LLM model based on chat history

with_message_history = RunnableWithMessageHistory(model,get_session_history)

In [ ]:
### Define a config parameter that defines the session id for a perticular session

### Here we have hardcoded our session id as 'chat1'

config = {'configurable' : {'session_id' : 'chat1'}}

In [ ]:
messages = [HumanMessage(content = "My name is Ashish and I am an DevOps engineer")]
with_message_history.invoke(messages, config=config)

In [ ]:
### Lets ask the chat model another question with respect to same session id

with_message_history.invoke([HumanMessage(content='What is my name and what I do?')], config=config)

In [ ]:
### Lets create a new sesson id

config_2 = {'configurable' : {'session_id' : 'chat2'}}

In [ ]:
messages = [HumanMessage(content='My name is Suresh and I am a CA')]
with_message_history.invoke(messages, config=config_2)

In [ ]:
### Lets ask the question to the chat model with respect to 'chat2' session id

with_message_history.invoke([HumanMessage(content='What is my name and what I do?')], config=config_2)

In [ ]:
### Lets create a new session id and ask the same question as previously asked

config_3 = {'configurable' : {'session_id' : 'chat3'}}

In [ ]:
with_message_history.invoke([HumanMessage(content='What is my name and what I do?')], config=config_3)